**Structured Output in LangChain**

Structured output means the language model returns results in a clear, organized format (like JSON or Python objects), not just plain text. This makes it easier to use and integrate model responses in applications.

**Pydantic:**
Pydantic is a Python library for data validation and settings management using Python type annotations.

In [9]:
from langchain.chat_models import init_chat_model

model = init_chat_model('groq:qwen/qwen3-32b')
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x0000025196D6AE90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000025196D6B890>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [9]:
from pydantic import BaseModel,Field

class Movie(BaseModel):
    title:str = Field(description="The title of the movie")
    year:int = Field(description="The release year of the movie")
    director:str = Field(description="The director of the movie")
    ratin:float = Field(description="The rating of the movie out of 10")
    actor:list[str] = Field(description="The main actors in the movie")

In [10]:
model_with_structure = model.with_structured_output(Movie)
model_with_structure.invoke("Provide details about the movie Breaking Bad.")

Movie(title='Breaking Bad', year=2008, director='Vince Gilligan', ratin=9.5, actor=['Bryan Cranston', 'Aaron Paul'])

In [17]:
from pydantic import BaseModel,Field

class Country(BaseModel):
    capital:str = Field(description="The capital of Country")
    indpendence_date:str=Field(description="The date of independence of the country")
    prime_minister:str=Field(description="The Current Prime Minister of the country")
    currencey:str=Field(description="The currency of the country")
    # wars_with_india:list[str]=Field(description="The wars fought between the country and India with dates")

country = model.with_structured_output(Country,include_raw = False)

country.invoke("Provide details about India.")

Country(capital='New Delhi', indpendence_date='August 15, 1947', prime_minister='Narendra Modi', currencey='Indian Rupee (INR)')

### Nested Structure

In [21]:
from pydantic import BaseModel, Field

class Actor(BaseModel):
    name:str
    role:str
class MovieDetail(BaseModel):
    title:str
    year:int
    cast:list[Actor]
    genres:list[str]
    budget:float | None = Field(None, description="The budget of the movie in USD")

nested = model.with_structured_output(MovieDetail,include_raw = False)

nested.invoke("Provide details about the movie Inception.")


MovieDetail(title='Inception', year=2010, cast=[Actor(name='Leonardo DiCaprio', role='Dom Cobb'), Actor(name='Joseph Gordon-Levitt', role='Arthur'), Actor(name='Ellen Page', role='Ariadne'), Actor(name='Tom Hardy', role='Jakob')], genres=['Action', 'Sci-Fi', 'Thriller'], budget=160000000.0)

# TypedDict
A TypedDict lets you specify the expected keys and value types for a dictionary

In [11]:
from typing_extensions import TypedDict,Annotated

class MovieDict(TypedDict):
    title: Annotated[str, ..., "The title of the movie"]
    year: Annotated[int, ..., "The release year of the movie"]
    director: Annotated[str, ..., "The director of the movie"]
    rating: Annotated[float, ..., "The rating of the movie out of 10"]

model_withtypedict = model.with_structured_output(MovieDict)

response = model_withtypedict.invoke("Provide details about the movie The Avengers.")
print(response)


{'director': 'Joss Whedon', 'rating': 8, 'title': 'The Avengers', 'year': 2012}


In [12]:
from pydantic import BaseModel, Field

class Actor(TypedDict):
    name:str
    role:str
class MovieDetail(TypedDict):
    title:str
    year:int
    cast:list[Actor]
    genres:list[str]
    budget:float | None = Field(None, description="The budget of the movie in USD")

nested = model.with_structured_output(MovieDetail,include_raw = False)

nested.invoke("Provide details about the movie Inception.")


{'budget': 160000000,
 'cast': [{'name': 'Leonardo DiCaprio', 'role': 'Dom Cobb'},
  {'name': 'Joseph Gordon-Levitt', 'role': 'Arthur'},
  {'name': 'Elliot Page', 'role': 'Ariadne'},
  {'name': 'Tom Hardy', 'role': 'Balthazar'}],
 'genres': ['Science Fiction', 'Action', 'Adventure'],
 'title': 'Inception',
 'year': 2010}

# DataClases
A dataclass in Python is a simple way to create classes for storing data, automatically generating methods like init and repr for you

In [22]:
from langchain.agents import create_agent
from pydantic import BaseModel, Field

class Info(BaseModel):
    """Contact information of a person"""
    name: str = Field(description="The name of the person")
    email: str = Field(description="The email of the person")
    phone: str = Field(description="The phone number of the person")

agent = create_agent(
    model='groq:qwen/qwen3-32b',
    response_format=Info
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "Extract contact info from: John Doe, john.doe@example.com, 123-456-7890"}]
})

# result
print(result["structured_response"])

name='John Doe' email='john.doe@example.com' phone='123-456-7890'


# Example with Dataclass

In [24]:
from dataclasses import dataclass, field
from langchain.agents import create_agent

@dataclass
class Info:
    """Contact information of a person"""
    name: str
    email: str
    phone: str 

agent = create_agent(
    model='groq:qwen/qwen3-32b',
    response_format=Info
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "Extract contact info from: John Doe, john.doe@example.com, 123-456-7890"}]
})

result["structured_response"]

Info(name='John Doe', email='john.doe@example.com', phone='123-456-7890')